**◆使い方**

1実行後にPlayItGGにログイン

2実行後にPlayItGGにてポートを選択


**◆GoogleDriveのファイルmemo**

Colab Notebooks

　　palworld(現行鯖)

　　palworld_BU(バックアップ鯖)

　　PalServer(本体)


In [ ]:
#playitgg,seleniumのインストールとユーザー作成
import os
from google.colab import drive

def setup_playit():
  print("Installing Playit...")
  os.system('curl -SsL https://playit-cloud.github.io/ppa/key.gpg | gpg --dearmor | sudo tee /etc/apt/trusted.gpg.d/playit.gpg >/dev/null')
  os.system('echo "deb [signed-by=/etc/apt/trusted.gpg.d/playit.gpg] https://playit-cloud.github.io/ppa/data ./" | sudo tee /etc/apt/sources.list.d/playit-cloud.list')
  os.system('sudo apt update')
  os.system('sudo apt install playit')
  !playit setup

setup_playit()
#drive.mount('/content/drive')

!apt-get update
!pip install selenium

# palworld ユーザを作成
!useradd -p $(perl -e 'print crypt("password03", "\$6\$salt03")') palworld

# NOPASSWD sudo を付与
!apt install sudo
!echo "palworld ALL=(ALL) NOPASSWD:ALL" | sudo tee /etc/sudoers.d/palworld

#サーバーファイルとcookieをユーザーにコピー
!cp -r "/content/drive/MyDrive/Colab Notebooks/palworld" "/home/palworld"
!cp -r "/content/drive/MyDrive/Hello" "/home"

#palworldユーザーが操作できるように権限を変更
!chmod -R 777 /home/palworld

Installing Playit...
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://playit-cloud.github.io/ppa/data ./ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.support.ui import Select
from selenium.common import exceptions

import subprocess as sp
import ctypes
import urllib3
import pickle
import threading
import time
import os

#リロード用クラス
class SeleniumColaboratory():
    def __init__(self, mode="1"):
        # FilleA path for auto-access
        self.access_path_2 = "https://colab.research.google.com/drive/1_mDPLD-DJcF7V77xIhYJjYpCIsXxshrW#scrollTo=8mjlcFOEphkg"
        web_url = 'https://colab.research.google.com/drive/1_mDPLD-DJcF7V77xIhYJjYpCIsXxshrW#scrollTo=8mjlcFOEphkg'#めんどくて、、、
        cookies_file = '/home/Hello/morokoshi.pkl'

        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')

        self.driver = webdriver.Chrome(options=options)
        cookies = pickle.load(open(cookies_file,'rb')) # クッキーを読み込む

        self.driver.get(web_url)
        for c in cookies: # クッキーを設定する
          if c["domain"] == ".google.com":
            self.driver.add_cookie(c)

        self.mode = str(mode)

    def click_runall(self):

        select_dropdown = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.ID,"runtime-menu-button")))
        select_dropdown.click()
        time.sleep(1)
        select_dropdown = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.ID,":1s")))
        select_dropdown.click()

    def click_change_runtime(self):#ここ弄ればreCAPTCHAもスルー出来そう
        # ランタイムクリック
        select_dropdown = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.ID,"runtime-menu-button")))
        select_dropdown.click()
        # リソースの表示クリック
        select_dropdown = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.ID,":2b")))
        select_dropdown.click()


    def auto_access(self, path):
        try:
            #新規タブを開いて更新処理
            self.driver.execute_script("window.open()") #make new tab
            self.driver.switch_to.window(self.driver.window_handles[-1]) #switch new tab
            self.driver.get(path)
            # ページの要素が全て読み込まれるまで待機
            WebDriverWait(self.driver, 60).until(EC.presence_of_all_elements_located)

            # 指定のURLにアクセスできているか確認(認証ページに飛ばされていないか確認)
            time.sleep(3)
            cookies_file = '/home/morokoshi.pkl'
            cookies = self.driver.get_cookies() # クッキーを取得する
            pickle.dump(cookies,open(cookies_file,'wb')) # クッキーを保存する

            !rm -rf "/content/drive/MyDrive/Hello/morokoshi.pkl"
            !cp -r "/home/morokoshi.pkl" "/content/drive/MyDrive/Hello/"

            cur_url = self.driver.current_url
            print(cur_url)
            print("title：", self.driver.title)
            self.click_change_runtime()
            time.sleep(30)

        except urllib3.exceptions.NewConnectionError as e:
            print(str(e))
            print("********Portal New connection timed out***********")
            time.sleep(30)

        except urllib3.exceptions.MaxRetryError as e:
            print(str(e))
            time.sleep(30)
            print("*********Portal Max tries exceeded************")

    def set_mode(self, mode):

        if mode == "None":
            self.mode = "1"
        elif mode == "GPU":
            self.mode = "2"
        elif mode == "TPU":
            self.mode == "3"
        else:
            self.mode = "1"

    def main(self):
        while True:
            self.set_mode("GPU")
            self.auto_access(self.access_path_2)
            # 15分ごとにチェック
            time.sleep(900)

# カスタムスレッドクラス（threading強制停止用）
class twe(threading.Thread):
    def __init__(self, group=None, target=None, name=None, args=(), kwargs={}):
        threading.Thread.__init__(self, group=group, target=target, name=name)
        self.args = args
        self.kwargs = kwargs
        return

    def run(self):
        self._target(*self.args, **self.kwargs)

    def get_id(self):
        if hasattr(self, '_thread_id'):
            return self._thread_id
        for id, thread in threading._active.items():
            if thread is self:
                return id

    # 強制終了させる関数
    def raise_exception(self):
        thread_id = self.get_id()
        resu = ctypes.pythonapi.PyThreadState_SetAsyncExc(ctypes.c_long(thread_id), ctypes.py_object(SystemExit))
        if resu > 1:
            ctypes.pythonapi.PyThreadState_SetAsyncExc(ctypes.c_long(thread_id), 0)
            print('Failure in raising exception')

def run_reload():
    SC = SeleniumColaboratory()
    SC.main()

def run_palworld_server():
    ! su palworld -c ./PalServer.sh

def run_playit():
    os.system('playit')

def run_buckup():
    i = 1
    while i < 12:
        time.sleep(1800)
        print(str(i)+"回目のセーブ")
        !rm -rf "/content/drive/MyDrive/Colab Notebooks/Palworld_Backup_Save/Saved/"
        !cp -r "/home/palworld/Pal/Saved" "/content/drive/MyDrive/Colab Notebooks/Palworld_Backup_Save/Saved/"
        i += 1

def run_save():
    #セーブデータの保存とバックアップ※前回開始時のデータは消えるので注意
    #開始時のファイルをBUフォルダに移動
    !rm -rf "/content/drive/MyDrive/Colab Notebooks/palworld_BU/"
    !cp -r "/content/drive/MyDrive/Colab Notebooks/palworld" "/content/drive/MyDrive/Colab Notebooks/palworld_BU"

    #終了時のファイルをドライブにコピー
    !rm -rf "/content/drive/MyDrive/Colab Notebooks/palworld/"
    !cp -r "/home/palworld" "/content/drive/MyDrive/Colab Notebooks/palworld"
    print("セーブが完了しました")

def run_server():
    palworld_thread = twe(name = 'Thread A', target = run_palworld_server)
    playit_thread = threading.Thread(target=run_playit)
    reload_thread = threading.Thread(target=run_reload)
    #playit_thread = twe(name = 'Thread B', target = run_playit)

    palworld_thread.start()
    playit_thread.start()
    reload_thread.start()

    run_buckup()

    time.sleep(1800)
    palworld_thread.raise_exception()

    palworld_thread.join()
    run_save()

%cd "/home/palworld"
#drive.mount('/content/drive')
#%cd "/content/drive/MyDrive/Colab Notebooks/palworld"
#!chmod -R 777 "/content/drive/MyDrive/Colab Notebooks/palworld"

run_server()

/home/palworld
The file already exists: /home/palworld/Pal/Binaries/Linux/steamclient.so
chmod: changing permissions of '/home/palworld/Pal/Binaries/Linux/PalServer-Linux-Shipping': Operation not permitted
Shutdown handler: initalize.
5.1.1-0+++UE5+Release-5.1 1008 0
Disabling core dumps.
sh: 1: xdg-user-dir: not found
[S_API] SteamAPI_Init(): Loaded local 'steamclient.so' OK.
CAppInfoCacheReadFromDiskThread took 0 milliseconds to initialize
Setting breakpad minidump AppID = 2394010
[S_API FAIL] Tried to access Steam interface SteamUser021 before SteamAPI_Init succeeded.
[S_API FAIL] Tried to access Steam interface SteamFriends017 before SteamAPI_Init succeeded.
[S_API FAIL] Tried to access Steam interface STEAMAPPS_INTERFACE_VERSION008 before SteamAPI_Init succeeded.
[S_API FAIL] Tried to access Steam interface SteamNetworkingUtils004 before SteamAPI_Init succeeded.
Game version is v0.3.10.61027
Running Palworld dedicated server on :8211
https://colab.research.google.com/drive/1_mDPLD

In [ ]:
#手動セーブ用
#セーブデータの保存とバックアップ※前回開始時のデータは消えるので注意
#開始時のファイルをBUフォルダに移動
!rm -rf "/content/drive/MyDrive/Colab Notebooks/palworld_BU/"
!cp -r "/content/drive/MyDrive/Colab Notebooks/palworld" "/content/drive/MyDrive/Colab Notebooks/palworld_BU"

#終了時のファイルをドライブにコピー
!rm -rf "/content/drive/MyDrive/Colab Notebooks/palworld/"
!cp -r "/home/palworld" "/content/drive/MyDrive/Colab Notebooks/palworld"